In [1]:
using JuMP

In [2]:
using Gurobi

In [3]:
using Plots

In [4]:
function pathCost(dist::Array{Float64,2}, path::Array{Int,1})
    n1, n2 = size(dist)
    @assert n1 == n2
    @assert 1 == minimum(path)
    @assert      maximum(path) == n1
    
    n = length(path)
    return sum(dist[path[i], path[i+1]] for i in 1:n-1)
end

pathCost (generic function with 1 method)

In [5]:
function vrptw(insta::String)
    # load instance
    include( joinpath(@__DIR__, insta) )
    c = [sqrt((X[i] - X[j])^2 + (Y[i] - Y[j])^2) for i in 1:n+2, j in 1:n+2]
    t = c .+ st
    
    nK = 25 # FIX
    BigM = maximum(b) + maximum(t) # FIX
    
    # vrptw model
    m = JuMP.Model()
    @variable(m, x[1:n+2, 1:n+2, 1:nK], Bin)
    @variable(m, s[1:n+2, 1:nK] >= 0.0)
    
    @objective(m, Min, sum(c[i,j] * x[i,j,k] for i in 1:n+2 for j in 1:n+2 for k in 1:nK))
    
    @constraint(m, r0[i in 1:n+2], sum(x[i, i, :]) == 0) # FIX
    @constraint(m, r1[i in 2:n+1], sum(x[i, :, :]) == 1)
    @constraint(m, r2[k in 1:nK], sum(d .* x[:, :, k]) <= q)
    @constraint(m, r3[k in 1:nK], sum(x[1, :, k]) == 1)
    @constraint(m, r4[h in 2:n+1, k in 1:nK], sum(x[:, h, k]) - sum(x[h, :, k]) == 0)
    @constraint(m, r5[k in 1:nK], sum(x[:, n+2, k]) == 1)
    @constraint(m, r6[i in 1:n+2, j in 1:n+2, k in 1:nK], s[i,k] + t[i,j] - BigM*(1 - x[i,j,k]) <= s[j,k])
    @constraint(m, r7[i in 1:n+2, k in 1:nK], a[i] <= s[i,k] <= b[i])
    @constraint(m, r8, sum(x[1, :, :]) <= nK)
    
    # set gurobi solver
    JuMP.set_optimizer(m, Gurobi.Optimizer)
    JuMP.set_optimizer_attribute(m, "LogToConsole", 0)
    JuMP.set_optimizer_attribute(m, "LogFile", joinpath(@__DIR__, "logfile.txt"))
    JuMP.set_optimizer_attribute(m, "TimeLimit", 600)  
    
    # optimize master model
    JuMP.optimize!(m)
    @assert JuMP.primal_status(m) == JuMP.FEASIBLE_POINT
    xval = JuMP.value.(x) .≈ 1.0
    
    # get the paths
    paths = []
    f = write_to_file
    for k in 1:nK
        path1 = Int[1]
        while true
            push!(path1, argmax(xval[path1[end], :, k]))
            if path1[end] == n+2
                break
            end
        end
        push!(paths, path1)
    end
    
    # write solution to file
    txtfile = "vrptw-" * insta[1:length(insta)-3] * ".txt"
    open(txtfile , "w") do io
        for k in 1:nK
            println(io, paths[k])
        end
        println(io, JuMP.objective_value(m))
    end

    return
end

vrptw (generic function with 1 method)

In [6]:
function vrptwcg(insta::String)
    # load instance
    include( joinpath(@__DIR__, insta) )
    c = [sqrt((X[i] - X[j])^2 + (Y[i] - Y[j])^2) for i in 1:n+2, j in 1:n+2]
    t = c .+ st
    
    # generate trivial solution
    paths = [ [1, i+1, n+2] for i in 1:n ]
    cp = [pathCost(c, paths[p]) for p in 1:n]
    
    # master model
    mm = JuMP.Model()
    @variable(mm, y[1:n] >= 0.0)
    @objective(mm, Min, sum(cp .* y))
    @constraint(mm, r1[i in 1:n], y[i] == 1)
    
    # set gurobi solver
    JuMP.set_optimizer(mm, Gurobi.Optimizer)
    JuMP.set_optimizer_attribute(mm, "LogToConsole", 0)
    JuMP.set_optimizer_attribute(mm, "LogFile", joinpath(@__DIR__, "logfile.txt"))
    JuMP.set_optimizer_attribute(mm, "TimeLimit", 600)

    # optimize master model
    JuMP.optimize!(mm)
    @assert JuMP.primal_status(mm) == JuMP.FEASIBLE_POINT
    
    # build slave model
    BigM = maximum(b) + maximum(t) # FIX
    ms = JuMP.Model()
    @variable(ms, x[1:n+2, 1:n+2], Bin)
    @variable(ms, s[1:n+2] >= 0.0)
    @objective(ms, Min, sum( (c .- 0.0) .* x))
    @constraint(ms, r00[i in 1:n+2], x[i, i] == 0) # FIX
    @constraint(ms, r01, sum(d .* x) <= q)
    @constraint(ms, r02, sum(x[1, :]) == 1)
    @constraint(ms, r03[h in 2:n+1], sum(x[:, h]) - sum(x[h, :]) == 0)
    @constraint(ms, r04, sum(x[:, n+2]) == 1)
    @constraint(ms, r05[i in 1:n+2, j in 1:n+2], s[i] + t[i,j] - BigM*(1 - x[i, j]) <= s[j])
    @constraint(ms, r06[i in 1:n+2], a[i] <= s[i] <= b[i])

    # set gurobi solver
    JuMP.set_optimizer(ms, Gurobi.Optimizer)
    JuMP.set_optimizer_attribute(ms, "LogToConsole", 0)
    JuMP.set_optimizer_attribute(ms, "LogFile", joinpath(@__DIR__, "logfile.txt"))
    JuMP.set_optimizer_attribute(ms, "TimeLimit", 600)
    
    # store the objective value mm and ms
    zmm::Vector{Float64} = []
    zms::Vector{Float64} = []
    
    while true
        # get dual variable / shadows price
        r1dual = [0.0; JuMP.dual.(r1); 0.0]

        # modify and re-optimize slave model
        @objective(ms, Min, sum( (c .- r1dual) .* x))
        JuMP.optimize!(ms)
        @assert JuMP.primal_status(ms) == JuMP.FEASIBLE_POINT
        xval = JuMP.value.(x) .≈ 1.0

        # get the path
        path1 = Int[1]
        while true
            push!(path1, argmax(xval[path1[end], :]))
            if path1[end] == n+2
                break
            end
        end

        # store the path and the cost
        push!(paths, path1)
        push!(cp, pathCost(c, path1))

        # insert the path in the master problem
        newY = @variable(mm; base_name = "y[$(length(y)+1)]", lower_bound = 0)
        push!(y, newY)
        JuMP.set_objective_coefficient(mm, newY, pathCost(c, path1))
        JuMP.set_normalized_coefficient.(r1, newY, [i in path1 for i in 2:n+1])

        # re-optimize master model
        JuMP.optimize!(mm)
        @assert JuMP.primal_status(mm) == JuMP.FEASIBLE_POINT
        yval = JuMP.value.(y)

        @show JuMP.objective_value(mm), JuMP.objective_value(ms)
        push!(zmm, JuMP.objective_value(mm))
        push!(zms, JuMP.objective_value(ms))

        if JuMP.objective_value(ms) >= -eps()
            break
        end        
    end
    
    # optimize the INTEGER master problem
    JuMP.set_integer.(y)
    JuMP.optimize!(mm)
    @assert JuMP.primal_status(mm) == JuMP.FEASIBLE_POINT
    yval = JuMP.value.(y)
    
    # plot master vs slave
    plot(zmm, label = "zMaster")
    plot!(twinx(), zms, label = "zSlave")
    pngfile = "vrptwCG-" * insta[1:length(insta)-3] * ".png"
    savefig(pngfile)
    
    # write solution to file
    txtfile = "vrptwCG-" * insta[1:length(insta)-3] * ".txt"
    open(txtfile , "w") do io
        for i in eachindex(yval)
            if yval[i] <= eps()
                continue
            end
            println(io, paths[i])
        end
        println(io, JuMP.objective_value(mm))
    end
    
    return
end

vrptwcg (generic function with 1 method)

In [7]:
instances = [
    "Solomon-R101.jl"
]

1-element Vector{String}:
 "Solomon-R101.jl"

In [8]:
for insta in instances
    #vrptwcg(insta)
    #vrptw(insta)
end

In [9]:
vrptw("Solomon-R101.jl")

Solomon-R101
Set parameter WLSAccessID
Set parameter WLSSecret
Set parameter LicenseID to value 780910
Academic license 780910 - for non-commercial use only - registered to ro___@linfati.cl


In [10]:
vrptwcg("Solomon-R101.jl")

Solomon-R101
Set parameter WLSAccessID
Set parameter WLSSecret
Set parameter LicenseID to value 780910
Academic license 780910 - for non-commercial use only - registered to ro___@linfati.cl
Set parameter WLSAccessID
Set parameter WLSSecret
Set parameter LicenseID to value 780910
Academic license 780910 - for non-commercial use only - registered to ro___@linfati.cl
(JuMP.objective_value(mm), JuMP.objective_value(ms)) = (4669.544753160757, -319.87786809696524)
(JuMP.objective_value(mm), JuMP.objective_value(ms)) = (4669.544753160757, -303.77823704826875)
(JuMP.objective_value(mm), JuMP.objective_value(ms)) = (4366.1422149542195, -303.4025382065362)
(JuMP.objective_value(mm), JuMP.objective_value(ms)) = (4062.980400457425, -303.1618144967951)
(JuMP.objective_value(mm), JuMP.objective_value(ms)) = (4062.980400457425, -303.07888833932475)
(JuMP.objective_value(mm), JuMP.objective_value(ms)) = (4062.980400457425, -289.6830562696385)
(JuMP.objective_value(mm), JuMP.objective_value(ms)) = (406